In [7]:
import torch
import pandas as pd
import re
import torch
import nltk

from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
from transformers import AutoModelForSequenceClassification
from transformers import set_seed

import clickhouse_connect

from sentence_transformers import SentenceTransformer
from FlagEmbedding import BGEM3FlagModel

from utils.utils import Chunker, Retriever
from utils.generators import GenerationConfig, Generator, SpellChecker, ToxicityClassifier

device = 'cuda'

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Соединение с БД

ClickHouse

In [8]:
client = clickhouse_connect.get_client(host='62.84.115.43', port=8123, username='viewer', password='viewer', database='dev')

# Подгрузка моделей

## Ретриверы

BGE-M3, E5-Large. Длина контекста ограничена нами 512 токенами

In [9]:
bge_retriever_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True, pooling_method="cls")
e5_retriever_model = SentenceTransformer("intfloat/multilingual-e5-large", device=device)
e5_retriever_model.max_seq_length = 512

loading existing colbert_linear and sparse_linear---------


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

## Генератор

FRED-T5-Large QA

In [10]:
generator_model_name = "hivaze/AAQG-QA-QG-FRED-T5-large"
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_model_name).to(device).eval()

tokenizer_config.json:   0%|          | 0.00/20.3k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.76M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.72k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

## Проверщик орфографии

T5-russian-spell. Используется для исправления неаккуратно написанных вопросов

In [11]:
spelchecker_model_name = 'UrukHan/t5-russian-spell'

spell_checker_tokenizer = T5TokenizerFast.from_pretrained(spelchecker_model_name)
spell_checker_model = AutoModelForSeq2SeqLM.from_pretrained(spelchecker_model_name).to(device).eval()

tokenizer_config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

## Классификатор Токсичного языка

RuBERT-Tiny. Используется для отклонения сообщений с матом или оскорблениями

In [12]:
toxicity_detection_model_name = 'cointegrated/rubert-tiny-toxicity'
toxicity_detection_tokenizer = AutoTokenizer.from_pretrained(toxicity_detection_model_name)
toxicity_detection_model = AutoModelForSequenceClassification.from_pretrained(toxicity_detection_model_name).to(device).eval()

tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

# Подгружаем подготовленные тестовые вопросы

In [15]:
book = pd.read_csv('book.csv', index_col=0)
book.head()

,url,query
0,https://cbr.ru/Crosscut/LawActs/File/6620/,Кто может принимать участие в закупках Банка Р...
1,https://cbr.ru/Queries/UniDbQuery/File/90134/2...,Cроки предоставления информации об активах и о...
2,https://cbr.ru/Queries/UniDbQuery/File/90002/12/,Что считается уровнем риска?
3,https://cbr.ru/Queries/UniDbQuery/File/90002/12/,Какими видами рисков обязательно должен управл...
4,https://cbr.ru/Crosscut/LawActs/File/6144/,На какие банковские операции распространяется ...


# Процесс Retrieval'a

## Инициализация классов

In [16]:
spell_checker = SpellChecker(spell_checker_tokenizer, spell_checker_model)
toxicity_detector = ToxicityClassifier(toxicity_detection_tokenizer, toxicity_detection_model)
retriever = Retriever(e5_retriever_model, bge_retriever_model, client)

In [17]:
generator_config = GenerationConfig(num_beams=5, num_return_sequences=5)
generator = Generator(generator_tokenizer, generator_model, config=generator_config)

## Получения семпла вопроса

In [18]:
question = book.iloc[7].query
question

'Какие полномочия у председателя ВЭБ.РФ?'

Сначала проверяем, нет ли оскорблений в тексте запроса, приводим его к орфографически правильному языку.

In [19]:
if toxicity_detector(question):
    raise ValueError

In [20]:
corrected_question = spell_checker(question)[0]
corrected_question

'Какие полномочия у председателя ВЭБ РФ?...'

Генерируем документ по вопросу (HyDE)

In [22]:
hyde_question = generator.hyde(question, temperature=0.5, num_beams=5)
hyde_question

'Полномочия председателя ВЭБ.РФ'

Идем в базу данных искать похожие на полученный документ документы. Возвращаем их текст и ссылки. Так как используются две модели retrieval'a, мы объединяем результаты поиска по пространствам обеих и подаем таблицу в Реранкер (BGE-M3 Colbert), который переранжирует документы.

In [23]:
topk_texts, topk_urls, question_embedding = retriever.get_neighbors(corrected_question, top_k=16)
rerank_indeces = retriever.rerank(question_embedding, topk_texts, 3)
reranked_texts, reranked_urls = retriever.get_topk(topk_texts, topk_urls, rerank_indeces)

На основе полученных документов и изначального вопроса генерируется ответ, также содержащий в себе ссылки на использованные файлы.

# Генерируем вопрос

In [24]:
print(generator(question, reranked_texts, reranked_urls, temperature=0.6, num_beams=5))

В соответствии с Федеральным законом от 22 апреля 1996 года No 39-ФЗ "О рынке ценных бумаг" председатель ВЭБ.РФ осуществляет следующие полномочия:
1) осуществляет руководство текущей деятельностью;
2) издает приказы и распоряжения по вопросам деятельности;3) утверждает организационную структуру и штатное расписание;4) определяет служебные права и обязанности работников;5) принимает решения по иным вопросам, отнесенным к компетенции наблюдательного совета или правления;6) назначает на должность своих заместителей, а также освобождает их от должности;7) распределяет обязанности между своими заместителями и иными работниками;8) выдает доверенности;9) устанавливает порядок осуществления отдельных видов банковских операций и сделок в случае противоречия порядка, установленного законодательством о банках и банковской деятельности, настоящему Федеральному закону;10) обеспечивает требования устойчивости и финансовой надежности кредитных организаций, соблюдения иных обязательных требований и но

## Бенчмарк (сколько раз мы верно угадали документ)

In [27]:
for i in [1, 3, 5, 10]:
    cur_recall = 0
    for ind, line in book.iterrows():
        url, question = line.url, line.query

        corrected_question = spell_checker(question)[0]
        hyde_question = generator.hyde(corrected_question, temperature=0.2, num_beams=5)
        topk_texts, topk_urls, question_embedding = retriever.get_neighbors(question, top_k=16, batch_size=16)
        rerank_indeces = retriever.rerank(question_embedding, topk_texts, top_k=i, batch_size=16)
        reranked_texts, reranked_urls = retriever.get_topk(topk_texts, topk_urls, rerank_indeces)
        cur_recall += url in reranked_urls

    print(f'recall@{i}:', cur_recall/len(book))

recall@1: 0.7333333333333333
recall@3: 0.9333333333333333
recall@5: 0.9333333333333333
recall@10: 0.9333333333333333


## Теперь попробуйте сами!

In [28]:
def generate(question, toxicity_detector, spell_checker, generator, retriever):
    if toxicity_detector(question):
        return 'Bad Question!'
    corrected_question = spell_checker(question)[0]
    hyde_question = generator.hyde(question, temperature=0.2, num_beams=5)
    topk_texts, topk_urls, question_embedding = retriever.get_neighbors(question, top_k=16)
    rerank_indeces = retriever.rerank(question_embedding, topk_texts, 3)
    reranked_texts, reranked_urls = retriever.get_topk(topk_texts, topk_urls, rerank_indeces)
    return generator(question, reranked_texts, reranked_urls, temperature=0.6, num_beams=5), reranked_texts

In [29]:
question = 'Какие полномочия у председателя ВЭБ РФ?'

In [30]:
answer, reranked_texts = generate(question, toxicity_detector, spell_checker, generator, retriever)
print(answer)

В соответствии с Федеральным законом от 22 апреля 1996 года No 39-ФЗ "О рынке ценных бумаг" председатель ВЭБ.РФ осуществляет следующие полномочия:
1) осуществляет руководство текущей деятельностью;
2) издает приказы и распоряжения по вопросам деятельности;3) утверждает организационную структуру и штатное расписание;4) определяет служебные права и обязанности работников;5) принимает решения по иным вопросам, отнесенным к компетенции наблюдательного совета или правления;6) назначает на должность своих заместителей, а также освобождает их от должности;7) распределяет обязанности между своими заместителями и иными работниками;8) выдает доверенности;9) устанавливает порядок осуществления отдельных видов банковских операций и сделок в случае противоречия порядка, установленного законодательством о банках и банковской деятельности, настоящему Федеральному закону;10) обеспечивает требования устойчивости и финансовой надежности кредитных организаций, соблюдения иных обязательных требований и но

### С использованными чанками докумнетов можно ознакомиться здесь

In [31]:
reranked_texts

['(В редакции Федерального закона от 28.11.2018 No 452-ФЗ)(Статья в редакции Федерального закона от 29.12.2017 No 454-ФЗ) Статья 15.\nПредседатель ВЭБ.РФ (Наименование в редакции Федерального закона от 28.11.2018 No 452-ФЗ) 1.\nПредседатель ВЭБ.РФ является единоличным исполнительным органом ВЭБ.РФ и осуществляет руководство его текущей деятельностью.\n(В редакции Федерального закона от 28.11.2018 No 452-ФЗ)2.\nПредседатель ВЭБ.РФ назначается на должность и освобождается от должности Президентом Российской Федерации.\nПредседатель ВЭБ.РФ назначается на должность по представлению Председателя Правительства Российской Федерации на срок не более пяти лет.\nКандидатура нового председателя ВЭБ.РФ представляется Президенту Российской Федерации за один месяц до дня истечения срока полномочий действующего председателя ВЭБ.РФ.\n(В редакции Федерального закона от 28.11.2018 No 452-ФЗ)3.\nВ случае отклонения Президентом Российской Федерации кандидатуры председателя ВЭБ.РФ председатель наблюдательн